<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Make_n_Save_Jacobians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data 

DOES NOT SUPPORT BATCH

Saves model JJ afterwords

Can handle: ResnetXXX with CIFAR10, CIFAR100, SVHN

i.e. CIFAR 10
```
 resnet20_cifar10
 resnet56_cifar10
 resnet110_cifar10
 resnet164bn_cifar10
 resnet272bn_cifar10

```

### TODO:  Add make and save full J also






In [54]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

modelnames = ['resnet20_cifar10','resnet56_cifar10','resnet110_cifar10','resnet164bn_cifar10', 'resnet272bn_cifar10']
num_classes = 10

num_data = 10000

device = 'cuda:0'


time: 1.73 ms


In [2]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 24kB/s 
     |████████████████████████████████| 6.6MB 15.4MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101


In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=e0dd933dc6a849e151d6006a3e4a3e954f2644363fd0a5822b582f03f49be88e
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 20.1 s


In [48]:
!ls -l "$Jdir"/*npy

ls: cannot access '/content/drive/My Drive/J_resnets_fullsvd/*npy': No such file or directory
time: 748 ms


In [15]:
!pip install pytorchcv
!pip install powerlaw
!pip install GPUtil
import GPUtil

     |████████████████████████████████| 440kB 2.8MB/s 
  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=107ebe13903159717b2f6f145708cb628abfd53b7d6f889bc3d4965cd0d2103b
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=482e207b444885f2978a1ed7b7505e049d21a59550cfbe7a3d7cb7ff161e3eaa
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
time: 9.53 s


In [16]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


/bin/bash: import: command not found
time: 1.11 s


### Run InstallPhysTheory notebook 

In [17]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 682 ms


In [26]:
def read_batched_JJMat(modelname, verbose=False):

  if verbose:
    print("Reading batched JJBlocks for ", modelname)
    
  JJMat = None
  import os, glob
  os.chdir(Jdir)
  for filename in glob.glob("Jfull_{}__*.csv".format(modelname)):
    if verbose:
      print(filename) 

    JJBlock = np.loadtxt(filename)
    JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock

  return JJMat

time: 4.01 ms


In [45]:
!ls "$Jdir"/*cifar10__50*csv

'/content/drive/My Drive/J_resnets_fullsvd/Jfull_resnet1001_cifar10__50.csv'
'/content/drive/My Drive/J_resnets_fullsvd/Jfull_resnet1202_cifar10__50.csv'
'/content/drive/My Drive/J_resnets_fullsvd/Jfull_resnet20_cifar10__50.csv'
'/content/drive/My Drive/J_resnets_fullsvd/Jfull_resnet542bn_cifar10__50.csv'
time: 1.05 s


In [49]:
def jacobian_full(model, data_loader, batch_size, num_classes=10, device='cuda:0'): 
  '''compute a full of J^{T}J Jacobian directly.  '''

  model.eval()
  model = model.to(device)

  J_right = jac.construct_diagJ(model, test_loader, batch_size=batch_size, device=device) 
  J_left = J_right.transpose(1,0)
  JJ = torch.mm(J_right, J_left)
  JJMat = JJ.detach().cpu().numpy()

  del J_right, J_left, JJ
  gc.collect()
  torch.cuda.empty_cache()

  return JJMat

time: 5.64 ms


### ResNet 

In [50]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 873 µs


### Compute JJ, in batches or all at once


In [51]:
def get_datasets(thismodel, batch_size):
  print("getting dataset for ", thismodel)
  if thismodel.endswith('cifar100'):
    print("loading cifar100")
    train_dataset = datasets.CIFAR100(
      root='data', 
      train=True, 
      transform=transforms.ToTensor(),
      download=True)
    test_dataset = datasets.CIFAR100(
      root='data', 
      train=False, 
      transform=transforms.ToTensor(),
      download=True) 
 
    train_loader = DataLoader(
      dataset=train_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)
    test_loader = DataLoader(
      dataset=test_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)

  else:
    print("loading cifar10")
    train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

      
  return train_dataset, test_dataset, train_loader, test_loader

time: 7.11 ms


In [52]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = get_datasets(thismodel, batch_size)#utils.get_data(batch_size=batch_size)
#train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

print(test_dataset)
GPUtil.showUtilization()


resnet56_cifar10
getting dataset for  resnet56_cifar10
loading cifar10
Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.4913725490196078, 0.4823529411764706, 0.4466666666666667], std=[0.24705882352941178, 0.24352941176470588, 0.2615686274509804])
           )
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 6.03 s


In [55]:
for modelname in modelnames:
  model = ptcv_get_model(modelname, pretrained=True)  
  JJMat = jacobian_full(model, test_loader, batch_size=batch_size,  num_classes=num_classes)
  filename = "{}/JJMat_{}".format(Jdir, modelname)
  print(filename)
  np.save(filename, JJMat)

/content/drive/My Drive/J_resnets_fullsvd/JJMat_resnet20_cifar10
/content/drive/My Drive/J_resnets_fullsvd/JJMat_resnet56_cifar10
/content/drive/My Drive/J_resnets_fullsvd/JJMat_resnet110_cifar10
/content/drive/My Drive/J_resnets_fullsvd/JJMat_resnet164bn_cifar10
/content/drive/My Drive/J_resnets_fullsvd/JJMat_resnet272bn_cifar10
time: 7min 54s
